# Comparison: L1 Minimization vs Polynomial Approximation for Imputation

This notebook compares two methods for recovering missing values in signals:

1. **L1 Minimization** (Theorem 1.20): Compressed sensing approach with L1 sparsity
2. **Polynomial Approximation** (Adaptive): L2 fitting with adaptive frequency selection

Both methods are tested on the same dataset with identical missing value patterns.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

# Import our modules
from fourier_core import fourier_ratio
from imputation import (
    mask_observations,
    compute_q,
    build_dft_basis,
    recover_l1_quadratic_constraint,
    recover_polynomial_approx,
    check_theorem_bound
)
from signal_utils import generate_composite_signal

## 1. Generate Test Signal and Simulate Missing Data

In [ ]:
# Signal parameters
sr = 32  # sampling rate (Hz)
seconds = 5.0  # duration
N = int(sr * seconds)

# Generate signal with some noise
t, f_full = generate_composite_signal(sr, seconds, noise_std=0.5)

# Compute Fourier Ratio
FR = fourier_ratio(f_full)
print(f"Signal length N = {N}")
print(f"Fourier Ratio FR = {FR:.4f}")
print(f"Interpretation: {'Low complexity (structured signal)' if FR < 5 else 'High complexity'}")

# Plot complete signal
plt.figure(figsize=(14, 4))
plt.plot(t, f_full, 'b-', label='Complete signal', linewidth=1.5)
plt.xlabel('Time (s)', fontsize=11)
plt.ylabel('Amplitude', fontsize=11)
plt.title('Original Complete Signal (with noise)', fontsize=13, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# Simulate missing data
keep_prob = 0.7  # Keep 70% of observations (30% missing)
seed = 42  # for reproducibility

mask, f_obs = mask_observations(f_full, keep_prob=keep_prob, seed=seed)

num_observed = np.sum(mask)
num_missing = np.sum(~mask)
missing_rate = num_missing / N * 100

print(f"Total samples: {N}")
print(f"Observed: {num_observed} ({100-missing_rate:.1f}%)")
print(f"Missing: {num_missing} ({missing_rate:.1f}%)")

# Visualize observed vs missing
plt.figure(figsize=(14, 5))
plt.plot(t, f_full, 'gray', alpha=0.3, label='True signal (hidden)', linewidth=2)
plt.scatter(t[mask], f_obs[mask], c='blue', s=15, label='Observed', zorder=3, alpha=0.7)
plt.scatter(t[~mask], f_full[~mask], c='red', s=25, label='Missing (ground truth)', 
            alpha=0.6, zorder=2, marker='x')
plt.xlabel('Time (s)', fontsize=11)
plt.ylabel('Amplitude', fontsize=11)
plt.title(f'Signal with {missing_rate:.1f}% Missing Values', fontsize=13, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 2. Method 1: L1 Minimization (Compressed Sensing)

This method solves an optimization problem:
$$\min \|\hat{c}\|_1 \text{ subject to } \|y - A\hat{c}\|_{2} \text{ is small}$$

where:
- $\hat{c}$ = Fourier coefficients
- $A$ = DFT basis evaluated at observed points
- $y$ = observed values
- L1 norm promotes sparsity in frequency domain

In [ ]:
# Parameters for L1 recovery
eps = 0.1  # desired recovery accuracy
C = 1.0  # universal constant multiplier

# Compute required observations
valid_idx = np.where(mask)[0]
q = compute_q(FR=FR, eps=eps, N=N, C=C, max_available=len(valid_idx))

print(f"=== L1 Minimization Method ===")
print(f"Recovery parameters:")
print(f"  ε (accuracy) = {eps}")
print(f"  Fourier Ratio = {FR:.4f}")
print(f"  Theoretical observations needed: q = {q}")
print(f"  Available observations: {len(valid_idx)}")

# Perform L1 recovery
print(f"\nPerforming L1 minimization...")
start_time = time.time()

# Select q observations
np.random.seed(seed)
obs_idx = np.random.choice(valid_idx, q, replace=False)
y = f_obs[obs_idx]

# Build DFT basis and measurement matrix
B = build_dft_basis(N)
A = B[obs_idx, :]

# Recover using L1 minimization
c_rec = recover_l1_quadratic_constraint(A, y, eps)
f_rec_l1 = B @ c_rec
f_rec_l1 = f_rec_l1.real  # Take real part

l1_time = time.time() - start_time
print(f"✓ L1 recovery completed in {l1_time:.3f} seconds")

## 3. Method 2: Polynomial Approximation (Adaptive Frequency Selection)

This method uses a simpler approach:
1. Project observed data onto ALL frequency basis functions
2. Select top-k frequencies with largest projection magnitudes
3. Refit using least squares (L2): $\min \|y - A_{selected}\hat{c}\|_2^2$
4. Evaluate on full grid

**Key difference**: L2 fitting (faster) vs L1 optimization (sparser)

In [ ]:
print(f"=== Polynomial Approximation Method ===")
print(f"Using same accuracy parameter: ε = {eps}")
print(f"\nPerforming polynomial approximation...")

start_time = time.time()
f_rec_poly = recover_polynomial_approx(f_obs, mask, eps=eps)
poly_time = time.time() - start_time

print(f"\n✓ Polynomial recovery completed in {poly_time:.3f} seconds")
print(f"\nSpeed comparison: Polynomial is {l1_time/poly_time:.1f}x faster than L1")

## 4. Error Metrics Comparison

In [ ]:
# Compute comprehensive error metrics for both methods

def compute_metrics(f_true, f_pred, mask, method_name):
    """Compute and display error metrics."""
    # Overall metrics
    mse_full = np.mean((f_true - f_pred) ** 2)
    rmse_full = np.sqrt(mse_full)
    mae_full = np.mean(np.abs(f_true - f_pred))
    rel_err_full = np.linalg.norm(f_true - f_pred) / np.linalg.norm(f_true)
    
    # Metrics on missing points only
    missing_mask = ~mask
    mse_missing = np.mean((f_true[missing_mask] - f_pred[missing_mask]) ** 2)
    rmse_missing = np.sqrt(mse_missing)
    mae_missing = np.mean(np.abs(f_true[missing_mask] - f_pred[missing_mask]))
    rel_err_missing = np.linalg.norm(f_true[missing_mask] - f_pred[missing_mask]) / np.linalg.norm(f_true[missing_mask])
    
    # Metrics on observed points only
    mse_obs = np.mean((f_true[mask] - f_pred[mask]) ** 2)
    rmse_obs = np.sqrt(mse_obs)
    mae_obs = np.mean(np.abs(f_true[mask] - f_pred[mask]))
    rel_err_obs = np.linalg.norm(f_true[mask] - f_pred[mask]) / np.linalg.norm(f_true[mask])
    
    print(f"\n{'='*60}")
    print(f"{method_name} - Error Metrics")
    print(f"{'='*60}")
    
    print(f"\n** Full Signal **")
    print(f"  RMSE:          {rmse_full:.6f}")
    print(f"  MAE:           {mae_full:.6f}")
    print(f"  Relative L2:   {rel_err_full:.6f}")
    
    print(f"\n** Missing Points Only **")
    print(f"  RMSE:          {rmse_missing:.6f}")
    print(f"  MAE:           {mae_missing:.6f}")
    print(f"  Relative L2:   {rel_err_missing:.6f}")
    
    print(f"\n** Observed Points Only **")
    print(f"  RMSE:          {rmse_obs:.6f}")
    print(f"  MAE:           {mae_obs:.6f}")
    print(f"  Relative L2:   {rel_err_obs:.6f}")
    
    return {
        'rmse_full': rmse_full,
        'mae_full': mae_full,
        'rel_err_full': rel_err_full,
        'rmse_missing': rmse_missing,
        'mae_missing': mae_missing,
        'rel_err_missing': rel_err_missing,
        'rmse_obs': rmse_obs,
        'mae_obs': mae_obs,
        'rel_err_obs': rel_err_obs
    }

# Compute metrics for both methods
metrics_l1 = compute_metrics(f_full, f_rec_l1, mask, "L1 Minimization")
metrics_poly = compute_metrics(f_full, f_rec_poly, mask, "Polynomial Approximation")

In [ ]:
# Comparison table
print(f"\n{'='*80}")
print(f"COMPARISON SUMMARY")
print(f"{'='*80}")
print(f"\n{'Metric':<30} {'L1 Minimization':<20} {'Polynomial Approx':<20} {'Winner'}")
print(f"{'-'*80}")

def compare_metric(name, val1, val2):
    if val1 < 1e-10 and val2 < 1e-10:
        winner = "Tie (both near-perfect)"
        ratio = 1.0
    elif val1 < val2:
        winner = "L1"
        ratio = val2/val1 if val1 > 1e-10 else float('inf')
    else:
        winner = "Polynomial"
        ratio = val1/val2 if val2 > 1e-10 else float('inf')
    
    if ratio == float('inf'):
        print(f"{name:<30} {val1:<20.6f} {val2:<20.6f} {winner} (perfect)")
    else:
        print(f"{name:<30} {val1:<20.6f} {val2:<20.6f} {winner} ({ratio:.2f}x)")

print("\nFull Signal:")
compare_metric("  RMSE", metrics_l1['rmse_full'], metrics_poly['rmse_full'])
compare_metric("  MAE", metrics_l1['mae_full'], metrics_poly['mae_full'])
compare_metric("  Relative L2 Error", metrics_l1['rel_err_full'], metrics_poly['rel_err_full'])

print("\nMissing Points Only:")
compare_metric("  RMSE", metrics_l1['rmse_missing'], metrics_poly['rmse_missing'])
compare_metric("  MAE", metrics_l1['mae_missing'], metrics_poly['mae_missing'])
compare_metric("  Relative L2 Error", metrics_l1['rel_err_missing'], metrics_poly['rel_err_missing'])

print("\nObserved Points Only:")
compare_metric("  RMSE", metrics_l1['rmse_obs'], metrics_poly['rmse_obs'])
compare_metric("  MAE", metrics_l1['mae_obs'], metrics_poly['mae_obs'])
compare_metric("  Relative L2 Error", metrics_l1['rel_err_obs'], metrics_poly['rel_err_obs'])

print(f"\nComputational Time:")
print(f"  L1 Minimization:         {l1_time:.3f} seconds")
print(f"  Polynomial Approximation: {poly_time:.3f} seconds")
print(f"  Speedup:                  {l1_time/poly_time:.1f}x")
print(f"{'='*80}")

## 5. Visual Comparison

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(3, 2, figsize=(16, 12))

# Plot 1: Full view - L1 Minimization
ax = axes[0, 0]
ax.plot(t, f_full, 'k-', alpha=0.4, label='True signal', linewidth=2)
ax.scatter(t[mask], f_obs[mask], c='blue', s=10, label='Observed', zorder=3, alpha=0.6)
ax.plot(t, f_rec_l1, 'r--', label='L1 recovery', linewidth=2, alpha=0.8)
ax.set_xlabel('Time (s)', fontsize=10)
ax.set_ylabel('Amplitude', fontsize=10)
ax.set_title(f'L1 Minimization (RMSE={metrics_l1["rmse_full"]:.4f})', fontsize=11, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Plot 2: Full view - Polynomial Approximation
ax = axes[0, 1]
ax.plot(t, f_full, 'k-', alpha=0.4, label='True signal', linewidth=2)
ax.scatter(t[mask], f_obs[mask], c='blue', s=10, label='Observed', zorder=3, alpha=0.6)
ax.plot(t, f_rec_poly, 'g--', label='Polynomial recovery', linewidth=2, alpha=0.8)
ax.set_xlabel('Time (s)', fontsize=10)
ax.set_ylabel('Amplitude', fontsize=10)
ax.set_title(f'Polynomial Approximation (RMSE={metrics_poly["rmse_full"]:.4f})', fontsize=11, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Plot 3: Missing points only - L1
ax = axes[1, 0]
missing_mask = ~mask
ax.scatter(t[missing_mask], f_full[missing_mask], c='black', s=30, 
           label='True (missing)', marker='o', alpha=0.6, zorder=3)
ax.scatter(t[missing_mask], f_rec_l1[missing_mask], c='red', s=30, 
           label='L1 recovered', marker='x', alpha=0.8, zorder=4)
ax.set_xlabel('Time (s)', fontsize=10)
ax.set_ylabel('Amplitude', fontsize=10)
ax.set_title(f'L1: Missing Points (MAE={metrics_l1["mae_missing"]:.4f})', fontsize=11, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Plot 4: Missing points only - Polynomial
ax = axes[1, 1]
ax.scatter(t[missing_mask], f_full[missing_mask], c='black', s=30, 
           label='True (missing)', marker='o', alpha=0.6, zorder=3)
ax.scatter(t[missing_mask], f_rec_poly[missing_mask], c='green', s=30, 
           label='Poly recovered', marker='x', alpha=0.8, zorder=4)
ax.set_xlabel('Time (s)', fontsize=10)
ax.set_ylabel('Amplitude', fontsize=10)
ax.set_title(f'Polynomial: Missing Points (MAE={metrics_poly["mae_missing"]:.4f})', fontsize=11, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Plot 5: Point-wise error - L1
ax = axes[2, 0]
error_l1 = np.abs(f_full - f_rec_l1)
ax.plot(t, error_l1, 'r-', alpha=0.5, linewidth=1)
ax.scatter(t[missing_mask], error_l1[missing_mask], c='red', s=20, 
           label=f'Error at missing (mean={np.mean(error_l1[missing_mask]):.4f})')
ax.axhline(y=np.mean(error_l1[missing_mask]), color='darkred', linestyle='--', 
           linewidth=2, alpha=0.7)
ax.set_xlabel('Time (s)', fontsize=10)
ax.set_ylabel('Absolute Error', fontsize=10)
ax.set_title('L1: Point-wise Absolute Error', fontsize=11, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Plot 6: Point-wise error - Polynomial
ax = axes[2, 1]
error_poly = np.abs(f_full - f_rec_poly)
ax.plot(t, error_poly, 'g-', alpha=0.5, linewidth=1)
ax.scatter(t[missing_mask], error_poly[missing_mask], c='green', s=20, 
           label=f'Error at missing (mean={np.mean(error_poly[missing_mask]):.4f})')
ax.axhline(y=np.mean(error_poly[missing_mask]), color='darkgreen', linestyle='--', 
           linewidth=2, alpha=0.7)
ax.set_xlabel('Time (s)', fontsize=10)
ax.set_ylabel('Absolute Error', fontsize=10)
ax.set_title('Polynomial: Point-wise Absolute Error', fontsize=11, fontweight='bold')
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Direct Side-by-Side Comparison

In [ ]:
# Side-by-side comparison on same plot
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Plot 1: Both methods overlaid
ax = axes[0]
ax.plot(t, f_full, 'k-', alpha=0.5, label='True signal', linewidth=2.5, zorder=1)
ax.scatter(t[mask], f_obs[mask], c='blue', s=15, label='Observed', zorder=4, alpha=0.5)
ax.plot(t, f_rec_l1, 'r--', label=f'L1 (RMSE={metrics_l1["rmse_full"]:.4f})', 
        linewidth=2, alpha=0.8, zorder=2)
ax.plot(t, f_rec_poly, 'g-.', label=f'Polynomial (RMSE={metrics_poly["rmse_full"]:.4f})', 
        linewidth=2, alpha=0.8, zorder=3)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.set_title('Both Methods Compared: Full Signal', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# Plot 2: Error comparison
ax = axes[1]
ax.plot(t, error_l1, 'r-', label=f'L1 error (mean={np.mean(error_l1):.4f})', 
        linewidth=2, alpha=0.7)
ax.plot(t, error_poly, 'g-', label=f'Polynomial error (mean={np.mean(error_poly):.4f})', 
        linewidth=2, alpha=0.7)
ax.scatter(t[missing_mask], error_l1[missing_mask], c='darkred', s=15, 
           label='L1 error at missing', alpha=0.6, zorder=3)
ax.scatter(t[missing_mask], error_poly[missing_mask], c='darkgreen', s=15, 
           label='Poly error at missing', alpha=0.6, zorder=3)
ax.set_xlabel('Time (s)', fontsize=12)
ax.set_ylabel('Absolute Error', fontsize=12)
ax.set_title('Error Comparison: Both Methods', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("SUMMARY: L1 Minimization vs Polynomial Approximation")
print("="*80)

print(f"\n**Signal Properties:**")
print(f"  Signal length:        {N}")
print(f"  Fourier Ratio:        {FR:.4f} (lower is more structured)")
print(f"  Missing rate:         {missing_rate:.1f}%")
print(f"  Accuracy parameter:   ε = {eps}")

print(f"\n**Method Characteristics:**")
print(f"\n  L1 Minimization (Theorem 1.20):")
print(f"    - Approach: Compressed sensing, L1 optimization")
print(f"    - Observations used: {q} out of {num_observed}")
print(f"    - Computation time: {l1_time:.3f} seconds")
print(f"    - Promotes sparsity via L1 norm")

print(f"\n  Polynomial Approximation (Adaptive L2):")
print(f"    - Approach: Adaptive frequency selection, L2 fitting")
print(f"    - All observations used: {num_observed}")
print(f"    - Computation time: {poly_time:.3f} seconds ({l1_time/poly_time:.1f}x faster)")
print(f"    - Direct least squares (no iterative optimization)")

print(f"\n**Accuracy Comparison (Missing Points):**")
winner_rmse = "L1" if metrics_l1['rmse_missing'] < metrics_poly['rmse_missing'] else "Polynomial"
winner_mae = "L1" if metrics_l1['mae_missing'] < metrics_poly['mae_missing'] else "Polynomial"
winner_rel = "L1" if metrics_l1['rel_err_missing'] < metrics_poly['rel_err_missing'] else "Polynomial"

print(f"  RMSE:        L1={metrics_l1['rmse_missing']:.6f}  Poly={metrics_poly['rmse_missing']:.6f}  Winner: {winner_rmse}")
print(f"  MAE:         L1={metrics_l1['mae_missing']:.6f}  Poly={metrics_poly['mae_missing']:.6f}  Winner: {winner_mae}")
print(f"  Relative L2: L1={metrics_l1['rel_err_missing']:.6f}  Poly={metrics_poly['rel_err_missing']:.6f}  Winner: {winner_rel}")

# Determine overall winner
l1_wins = sum([
    metrics_l1['rmse_missing'] < metrics_poly['rmse_missing'],
    metrics_l1['mae_missing'] < metrics_poly['mae_missing'],
    metrics_l1['rel_err_missing'] < metrics_poly['rel_err_missing']
])

print(f"\n**Recommendation:**")
if l1_wins >= 2:
    print(f"  → L1 Minimization is MORE ACCURATE (wins {l1_wins}/3 metrics)")
    print(f"  → But Polynomial is {l1_time/poly_time:.1f}x FASTER")
    print(f"\n  Use L1 if accuracy is critical, Polynomial for speed")
else:
    print(f"  → Polynomial Approximation is MORE ACCURATE (wins {3-l1_wins}/3 metrics)")
    print(f"  → AND {l1_time/poly_time:.1f}x FASTER")
    print(f"\n  Recommendation: Use Polynomial Approximation")

print(f"\n**Key Insight:**")
print(f"  The adaptive polynomial method selects frequencies that best")
print(f"  explain the observed data, leading to excellent reconstruction")
print(f"  for structured signals with low Fourier Ratio.")

print("\n" + "="*80)

## 8. Verify Theoretical Bounds

Check if L1 minimization satisfies Theorem 1.20 bound.

In [ ]:
# Check Theorem 1.20 bound for L1 method
lhs, rhs, ratio, ok = check_theorem_bound(f_full, f_rec_l1, eps)

print("\n=== Theorem 1.20 Bound Verification (L1 Method) ===")
print(f"||x* - f||₂         = {lhs:.6f}")
print(f"11.47 × ||f||₂ × ε  = {rhs:.6f}")
print(f"Ratio (lhs/rhs)     = {ratio:.4f}")
print(f"\nBound holds: {'✅ Yes' if ok else '❌ No'}")
if ok:
    print("The L1 recovery satisfies the theoretical guarantee!")
else:
    print("Try adjusting ε, C, or keep_prob parameters.")